Importing Libraries

In [3]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as mlt

import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

Defining Transformations

In [4]:
# For Training: Augmentation (helps to prevent overfitting) + Normalization
trainTransforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(), # Convert to Tensor (Pytorch uses tensor to store data for model.)
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# For test and validation:
valTransforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

Loding the Dataset

In [5]:
dataset = datasets.ImageFolder(root="D:\cell_images", transform=trainTransforms)
# dataset = datasets.ImageFolder(root="", transform=trainTransforms)

Spliting the data into Train and Validation (80%-20%)

In [6]:
trainSize = int(0.8*len(dataset))
valSize = len(dataset) - trainSize
trainDataset, valDataset = random_split(dataset, [trainSize, valSize])

valDataset.dataset.transform = valTransforms

Creating a DataLoaders

In [7]:
tainLoader = DataLoader(trainDataset, batch_size=32, shuffle=True)
valLoader = DataLoader(valDataset, batch_size=32, shuffle=False)